# Model Comparison and Analysis
## Mall Movement Tracking - Comprehensive Model Comparison

This notebook compares all trained models across different tasks:
- Classification models comparison
- Clustering models comparison
- Forecasting models comparison
- Overall best model selection
- Visualization and insights


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import sys
import json
import joblib

# Add project root to path
project_root = Path().resolve()
if project_root.name == 'notebooks':
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Project root: {project_root}")


## 1. Load Model Results


In [ ]:
# Load all results
RESULTS_DIR = project_root / "results"

# Classification results
with open(RESULTS_DIR / "classification" / "metrics.json", 'r') as f:
    classification_results = json.load(f)

# Clustering results
with open(RESULTS_DIR / "clustering" / "silhouette_score.json", 'r') as f:
    clustering_results = json.load(f)

# Forecasting results
try:
    with open(RESULTS_DIR / "forecasting" / "rmse.json", 'r') as f:
        forecasting_results = json.load(f)
except:
    forecasting_results = {}

print("Classification Results:")
print(json.dumps(classification_results, indent=2))

print("\nClustering Results:")
print(json.dumps(clustering_results, indent=2))

if forecasting_results:
    print("\nForecasting Results:")
    print(json.dumps(forecasting_results, indent=2))


## 2. Classification Models Comparison


In [ ]:
# Prepare classification comparison data
classification_df = pd.DataFrame([
    {
        'Model': 'Random Forest',
        'Accuracy': classification_results['random_forest']['accuracy'],
        'ROC-AUC': classification_results['random_forest'].get('roc_auc', np.nan),
        'Type': 'Ensemble'
    },
    {
        'Model': 'Decision Tree',
        'Accuracy': classification_results['decision_tree']['accuracy'],
        'ROC-AUC': np.nan,
        'Type': 'Baseline'
    },
    {
        'Model': 'XGBoost',
        'Accuracy': classification_results['xgboost']['accuracy'],
        'ROC-AUC': classification_results['xgboost'].get('roc_auc', np.nan),
        'Type': 'Gradient Boosting'
    }
])

print("Classification Models Comparison:")
print("=" * 60)
display(classification_df)

# Find best model
best_classification = classification_df.loc[classification_df['Accuracy'].idxmax()]
print(f"\n🏆 Best Classification Model: {best_classification['Model']}")
print(f"   Accuracy: {best_classification['Accuracy']:.4f} ({best_classification['Accuracy']*100:.2f}%)")


In [ ]:
# Visualize classification comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Accuracy comparison
axes[0].bar(classification_df['Model'], classification_df['Accuracy'], 
           color=['steelblue', 'coral', 'lightgreen'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[0].set_title('Classification Models - Accuracy Comparison', fontsize=14, fontweight='bold', pad=20)
axes[0].set_ylim([0.95, 1.0])
axes[0].grid(True, alpha=0.3, axis='y')
for i, (model, acc) in enumerate(zip(classification_df['Model'], classification_df['Accuracy'])):
    axes[0].text(i, acc + 0.002, f'{acc:.4f}', ha='center', fontweight='bold', fontsize=11)
    if acc == best_classification['Accuracy']:
        axes[0].text(i, acc - 0.008, '🏆 BEST', ha='center', fontweight='bold', fontsize=10, color='red')

# Model types
axes[1].barh(classification_df['Model'], classification_df['Accuracy'], 
            color=['steelblue', 'coral', 'lightgreen'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_xlabel('Accuracy', fontsize=12, fontweight='bold')
axes[1].set_title('Classification Models - Horizontal View', fontsize=14, fontweight='bold', pad=20)
axes[1].set_xlim([0.95, 1.0])
axes[1].grid(True, alpha=0.3, axis='x')
for i, (model, acc) in enumerate(zip(classification_df['Model'], classification_df['Accuracy'])):
    axes[1].text(acc + 0.001, i, f'{acc:.4f}', va='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig(RESULTS_DIR / "comparisons" / "classification_comparison.png", dpi=300, bbox_inches='tight')
print("✓ Saved: classification_comparison.png")
plt.show()


## 3. Clustering Models Comparison


In [ ]:
# Prepare clustering comparison data
clustering_df = pd.DataFrame([
    {
        'Model': 'K-Means',
        'Silhouette Score': clustering_results['kmeans']['silhouette_score'],
        'Number of Clusters': clustering_results['kmeans']['n_clusters'],
        'Noise Points': 0,
        'Type': 'Centroid-based'
    },
    {
        'Model': 'DBSCAN',
        'Silhouette Score': clustering_results['dbscan'].get('silhouette_score', np.nan),
        'Number of Clusters': clustering_results['dbscan']['n_clusters'],
        'Noise Points': clustering_results['dbscan']['n_noise'],
        'Type': 'Density-based'
    }
])

print("Clustering Models Comparison:")
print("=" * 60)
display(clustering_df)

# Find best model
best_clustering = clustering_df.loc[clustering_df['Silhouette Score'].idxmax()]
print(f"\n🏆 Best Clustering Model: {best_clustering['Model']}")
print(f"   Silhouette Score: {best_clustering['Silhouette Score']:.4f}")
print(f"   Number of Clusters: {best_clustering['Number of Clusters']}")


In [ ]:
# Visualize clustering comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Silhouette score comparison
valid_scores = clustering_df[clustering_df['Silhouette Score'].notna()]
axes[0].bar(valid_scores['Model'], valid_scores['Silhouette Score'], 
           color=['steelblue', 'coral'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Silhouette Score', fontsize=12, fontweight='bold')
axes[0].set_title('Clustering Models - Silhouette Score Comparison', fontsize=14, fontweight='bold', pad=20)
axes[0].set_ylim([0, 0.3])
axes[0].grid(True, alpha=0.3, axis='y')
for i, (model, score) in enumerate(zip(valid_scores['Model'], valid_scores['Silhouette Score'])):
    axes[0].text(i, score + 0.01, f'{score:.4f}', ha='center', fontweight='bold', fontsize=11)
    if score == best_clustering['Silhouette Score']:
        axes[0].text(i, score - 0.02, '🏆 BEST', ha='center', fontweight='bold', fontsize=10, color='red')

# Number of clusters comparison
axes[1].bar(clustering_df['Model'], clustering_df['Number of Clusters'], 
           color=['steelblue', 'coral'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_ylabel('Number of Clusters', fontsize=12, fontweight='bold')
axes[1].set_title('Clustering Models - Number of Clusters', fontsize=14, fontweight='bold', pad=20)
axes[1].grid(True, alpha=0.3, axis='y')
for i, (model, n_clusters) in enumerate(zip(clustering_df['Model'], clustering_df['Number of Clusters'])):
    axes[1].text(i, n_clusters + max(clustering_df['Number of Clusters']) * 0.02, 
                f'{int(n_clusters)}', ha='center', fontweight='bold', fontsize=11)
    if clustering_df.loc[i, 'Noise Points'] > 0:
        axes[1].text(i, n_clusters / 2, f'Noise: {int(clustering_df.loc[i, "Noise Points"])}', 
                    ha='center', fontsize=9, color='gray')

plt.tight_layout()
plt.savefig(RESULTS_DIR / "comparisons" / "clustering_comparison.png", dpi=300, bbox_inches='tight')
print("✓ Saved: clustering_comparison.png")
plt.show()


## 4. Forecasting Models Comparison


In [ ]:
# Prepare forecasting comparison data
if forecasting_results:
    forecasting_df = pd.DataFrame([
        {
            'Model': 'ARIMA',
            'RMSE': forecasting_results.get('arima', {}).get('rmse', np.nan),
            'MAE': forecasting_results.get('arima', {}).get('mae', np.nan),
            'Type': 'Statistical'
        },
        {
            'Model': 'Prophet',
            'RMSE': forecasting_results.get('prophet', {}).get('rmse', np.nan),
            'MAE': forecasting_results.get('prophet', {}).get('mae', np.nan),
            'Type': 'Facebook Prophet'
        }
    ])
    
    print("Forecasting Models Comparison:")
    print("=" * 60)
    display(forecasting_df)
    
    # Find best model (lower RMSE is better)
    valid_rmse = forecasting_df[forecasting_df['RMSE'].notna()]
    if len(valid_rmse) > 0:
        best_forecasting = valid_rmse.loc[valid_rmse['RMSE'].idxmin()]
        print(f"\n🏆 Best Forecasting Model: {best_forecasting['Model']}")
        print(f"   RMSE: {best_forecasting['RMSE']:.4f}")
        print(f"   MAE: {best_forecasting['MAE']:.4f}")
else:
    print("Forecasting results not available")
    forecasting_df = pd.DataFrame()


In [ ]:
# Visualize forecasting comparison (if available)
if len(forecasting_df) > 0 and forecasting_df['RMSE'].notna().any():
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # RMSE comparison (lower is better)
    valid_rmse = forecasting_df[forecasting_df['RMSE'].notna()]
    axes[0].bar(valid_rmse['Model'], valid_rmse['RMSE'], 
               color=['steelblue', 'coral'], alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[0].set_ylabel('RMSE', fontsize=12, fontweight='bold')
    axes[0].set_title('Forecasting Models - RMSE Comparison (Lower is Better)', fontsize=14, fontweight='bold', pad=20)
    axes[0].grid(True, alpha=0.3, axis='y')
    for i, (model, rmse) in enumerate(zip(valid_rmse['Model'], valid_rmse['RMSE'])):
        axes[0].text(i, rmse + max(valid_rmse['RMSE']) * 0.02, f'{rmse:.2e}', 
                    ha='center', fontweight='bold', fontsize=10)
    
    # MAE comparison
    valid_mae = forecasting_df[forecasting_df['MAE'].notna()]
    axes[1].bar(valid_mae['Model'], valid_mae['MAE'], 
               color=['steelblue', 'coral'], alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[1].set_ylabel('MAE', fontsize=12, fontweight='bold')
    axes[1].set_title('Forecasting Models - MAE Comparison (Lower is Better)', fontsize=14, fontweight='bold', pad=20)
    axes[1].grid(True, alpha=0.3, axis='y')
    for i, (model, mae) in enumerate(zip(valid_mae['Model'], valid_mae['MAE'])):
        axes[1].text(i, mae + max(valid_mae['MAE']) * 0.02, f'{mae:.2e}', 
                    ha='center', fontweight='bold', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / "comparisons" / "forecasting_comparison.png", dpi=300, bbox_inches='tight')
    print("✓ Saved: forecasting_comparison.png")
    plt.show()
else:
    print("Forecasting visualization skipped (insufficient data)")
